# SNS SCRAPE

### Install SNS Scrape

In [1]:
!pip install snscrape

### Setup SNS Scrape for our Tweets

In [2]:
import pandas as pd

df = pd.read_csv('../raw_data/artist_data.csv.csv')

In [3]:
df.head()

,ARTIST,DATE OF CANCELLATION,GENDER,RACE,GENRE,LEVEL FAME,REASON FOR CANCELLATION,TWITTER HANDLE,TWITTER FOLLOWERS,CHARTMETRIC ID,ARTIST NICKNAME,NICKNAME,CANCELLED
0,Nicki Minaj,2021-09-13,Female,Black,Hip-hop,Superstar,Spreading COVID misinformation,@NICKIMINAJ,26000000,3442,Nicki Minaj,"@NICKIMINAJ, Nicki Minaj",cancelled
1,Taylor Swift,2022-08-01,Female,White,Pop,Superstar,No.1 Private jet usage,@taylorswift13,91000000,2762,Taylor Swift,"@taylorswift13, Taylor Swift",cancelled
2,DaBaby,2021-07-26,Male,Black,Hip-hop,Superstar,Homophobic remarks/insensitive HIV comments,@DaBabyDaBaby,4500000,398544,DaBaby´,"@DaBabyDaBaby, DaBaby´",cancelled
3,Zayn Malik,2021-10-28,Male,South Asian,Pop,Superstar,Accused of physical assault,@zaynmalik,31000000,207124,"Zayn, Zayn Malik","@zaynmalik, Zayn, Zayn Malik",cancelled
4,Doja Cat,2020-05-22,Female,Black,Hip-hop,Superstar,Accused of affiliating with white supremacists,@DojaCat,5400000,217671,"Doja, Doja Cat","@DojaCat, Doja, Doja Cat",cancelled


In [4]:
df['NICKNAME'] = df['NICKNAME'].apply(lambda x : x.split(", "))

In [5]:
df

,ARTIST,DATE OF CANCELLATION,GENDER,RACE,GENRE,LEVEL FAME,REASON FOR CANCELLATION,TWITTER HANDLE,TWITTER FOLLOWERS,CHARTMETRIC ID,ARTIST NICKNAME,NICKNAME,CANCELLED
0,Nicki Minaj,2021-09-13,Female,Black,Hip-hop,Superstar,Spreading COVID misinformation,@NICKIMINAJ,26000000,3442,Nicki Minaj,"[@NICKIMINAJ, Nicki Minaj]",cancelled
1,Taylor Swift,2022-08-01,Female,White,Pop,Superstar,No.1 Private jet usage,@taylorswift13,91000000,2762,Taylor Swift,"[@taylorswift13, Taylor Swift]",cancelled
2,DaBaby,2021-07-26,Male,Black,Hip-hop,Superstar,Homophobic remarks/insensitive HIV comments,@DaBabyDaBaby,4500000,398544,DaBaby´,"[@DaBabyDaBaby, DaBaby´]",cancelled
3,Zayn Malik,2021-10-28,Male,South Asian,Pop,Superstar,Accused of physical assault,@zaynmalik,31000000,207124,"Zayn, Zayn Malik","[@zaynmalik, Zayn, Zayn Malik]",cancelled
4,Doja Cat,2020-05-22,Female,Black,Hip-hop,Superstar,Accused of affiliating with white supremacists,@DojaCat,5400000,217671,"Doja, Doja Cat","[@DojaCat, Doja, Doja Cat]",cancelled
5,Marilyn Manson,2021-02-01,Male,White,Rock,Superstar,Relationship abuse & domestic violence,@marilynmanson,1100000,2286,Marilyn Manson,"[@marilynmanson, Marilyn Manson]",cancelled
6,Travis Scott,2021-11-01,Male,Black,Hip-hop,Superstar,Astroworld festival crush,@trvisXX,11600000,4215,Travis Scott,"[@trvisXX, Travis Scott]",cancelled
7,Chris Brown,2022-01-28,Male,Black,Pop,Superstar,Rape & drug (Crime),@chrisbrown,32200000,2249,Chris Brown,"[@chrisbrown, Chris Brown]",cancelled
8,Lana Del Rey,2020-05-21,Female,White,Pop,Superstar,Accused of racism,NaN,NaN,3986,"Lana, Lana Del Rey","[Lana, Lana Del Rey]",cancelled
9,Justin Timberlake,2021-02-01,Male,White,Pop,Superstar,Britney Spears and Janet Jackson's confession,@jtimberlake,63000000,1450,Justin Timberlake,"[@jtimberlake, Justin Timberlake]",cancelled


In [6]:
import snscrape.modules.twitter as sntwitter
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd


def run_the_tweet(keywords,incident_date, n_months, min_retweets, language):
    date_format = '%Y-%m-%d'
    dtObj = datetime.strptime(incident_date, date_format)
    
    begin_date = (dtObj - relativedelta(months=n_months)).date()
    since_date = (dtObj - relativedelta(months=n_months)).date()
    
    until_date = since_date + timedelta(days=1)
    final_date = (dtObj + relativedelta(months=n_months)).date()
    
    n_days = (final_date-since_date).days
    
    query = " OR ".join(keywords)
    if len(keywords) == 1:
        query = keywords[0]
    tweets_list= []
    for x in range(n_days): #no. of days
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'{query} since:{since_date.strftime(date_format)} until:{until_date.strftime(date_format)}  min_retweets:{min_retweets} lang:{language}').get_items()):
            if i > 1: #no. of output tweet
                break
            else:
                tweets_list.append([tweet.date, tweet.retweetCount, tweet.likeCount, tweet.content, tweet.user.username]) #append if statement satisfy
        if x < n_days : # no. of days you want to be return
            since_date = since_date + timedelta(days=1) #add another day
            until_date = until_date + timedelta(days=1) #add another day
        else:
            break
    tweets_df = pd.DataFrame(tweets_list, columns=['Datetime',"retweet","likes", 'Text', 'Username'])
    return tweets_df


In [7]:
%%time
df = run_the_tweet(df["NICKNAME"][1], "2021-03-16" ,1, "5", "en")

CPU times: user 1.5 s, sys: 35.6 ms, total: 1.54 s
Wall time: 57.6 s


In [8]:
df

,Datetime,retweet,likes,Text,Username
0,2021-02-16 23:59:43+00:00,6,132,my therapist asked me what I tweeted about and...,blesstherain13
1,2021-02-16 23:53:22+00:00,12,384,"Stars, do u like dem⭐️✨ @taylorswift13 @taylor...",Dantasyland
2,2021-02-17 23:58:42+00:00,13,42,📊 | Youtube Trending (Overall) :\n\n15. #LoveS...,TSwiftVS
3,2021-02-17 23:51:05+00:00,18,127,Taylor Swift pose https://t.co/l8gQnGK6T2,vickidlover
4,2021-02-18 23:59:26+00:00,159,830,"HAIM's ""Gasoline"" remix feat. Taylor Swift is ...",TChartSwift
...,...,...,...,...,...
113,2021-04-13 23:45:04+00:00,185,754,folklore - taylor swift https://t.co/wEBJBLUohB,beepopofc
114,2021-04-14 23:56:00+00:00,7,19,"all women do is listen to all time low, be obs...",popsmenu
115,2021-04-14 23:51:55+00:00,6,54,WHY IS THIS SONG SO GOOD/GIVING ME SOFT TAYLOR...,swiftbunnies
116,2021-04-15 23:52:36+00:00,6,12,taylor swift as cats: a serotonin filled thread ♡,comersmcu


### Clean Tweets

In [9]:
!pip install nltk

In [10]:
!pip install emot

In [11]:
!pip install emoji

In [12]:
pip install clean-text

Note: you may need to restart the kernel to use updated packages.


In [13]:
import re
import string
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import emoji
from cleantext import clean


import string
import re

from emot.emo_unicode import UNICODE_EMOJI

def clean_text(text):
    #changing to lowercase
    text = text.lower()
    
    # removing #´s 
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'#', '', text)
    
    #remove RT
    text = re.sub(r'RT[\s]+', '', text)
    
    #remove links
    text = re.sub(r'https?:\/\/\S+', '', text)
    text = re.sub(r"www.\S+", "", text)
    
    #remove indentation
    text = re.sub(r'\n', '', text)

    #remove emojis
    text = clean(text, no_emoji=True)
    
    
    #tokenizing words
    tokens = word_tokenize(text)
    
    filtered_words = [w for w in tokens if w not in list(UNICODE_EMOJI.keys())]
    
    return text

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/malaiafiq/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/malaiafiq/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


***
# SENTIMENT ANALYSIS

In [14]:
!pip install transformers

In [15]:
!pip install torch

In [16]:
pip install ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

In [ ]:
from transformers import pipeline

sentiment_task = pipeline("sentiment-analysis", model= model, tokenizer= tokenizer, top_k=3)

In [ ]:
df['cleaned_tweet'] = df.Text.apply(clean_text)

In [ ]:
df

In [ ]:
def get_sentiment_score_dictionary(df_cleanedtext):
    list_sentiments = []
    for tweet in df.cleaned_tweet:
        sentiment_task_result = sentiment_task(tweet)[0]
        tweet_sentiment = {}
        for label in sentiment_task_result:
            if label['label'] == 'Neutral':
                tweet_sentiment['Neutral'] = label['score']
            elif label['label'] == 'Positive':
                tweet_sentiment['Positive'] = label['score']
            else:
                tweet_sentiment['Negative'] = label['score']
        list_sentiments.append(tweet_sentiment)
    return list_sentiments

def get_sentiment_score_classification(list_sentiments):
    sentiment_classifications = []
    for list_sentiment in list_sentiments:
        if (list_sentiment['Positive'] > list_sentiment['Negative']) and (list_sentiment['Positive'] > list_sentiment['Neutral']):
            sentiment_classifications.append('Positive')
        elif (list_sentiment['Neutral'] > list_sentiment['Negative']) and (list_sentiment['Positive'] < list_sentiment['Neutral']):
            sentiment_classifications.append('Neutral')
        else:
            sentiment_classifications.append('Negative')
    return sentiment_classifications

In [ ]:
classified_sentiment_ts = get_sentiment_score_classification(get_sentiment_score_dictionary(df.cleaned_tweet))

In [ ]:
pd.merge(df, pd.DataFrame.from_dict(get_sentiment_score_dictionary(df.cleaned_tweet)), left_index= True, right_index= True)

In [ ]:
ts_df = pd.merge(df, pd.DataFrame.from_dict(classified_sentiment_ts), left_index= True, right_index= True)

In [ ]:
ts_df.rename(columns = {0:'sentiment_classification'}, inplace = True)

In [ ]:
ts_df.sentiment_classification.value_counts()